# Sentiment analysis

In [28]:
import pandas as pd
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

# Data Preprocessing

In [29]:
tweets_df = pd.read_csv("data/stock_tweets.csv")

# Text cleanup
def clean_tweet(text):
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub("\\r" ,'',text)
    text = re.sub("\\n" ,'',text)
    # Remove emojis using Unicode ranges
    text = re.sub(r'[\U0001F600-\U0001F64F'
                  r'\U0001F300-\U0001F5FF'
                  r'\U0001F680-\U0001F6FF'
                  r'\U0001F700-\U0001F77F'
                  r'\U0001F780-\U0001F7FF'
                  r'\U0001F800-\U0001F8FF'
                  r'\U0001F900-\U0001F9FF'
                  r'\U0001FA00-\U0001FA6F'
                  r'\U0001FA70-\U0001FAFF'
                  r'\U00002702-\U000027B0'
                  r'\U000024C2-\U0001F251]', '', text)
    return text.strip()

# Apply the function to the "tweet" column
tweets_df['Tweet'] = tweets_df['Tweet'].apply(clean_tweet)

tweets_df.sample(n=5)

,Date,Tweet,Stock Name,Company Name
41351,2021-10-25 13:27:10+00:00,MS PB says Hedge Fund net exposure to Large Ca...,MSFT,Microsoft Corporation
19705,2022-03-03 08:22:05+00:00,. i just bought a but the engine is missing. ...,TSLA,"Tesla, Inc."
15323,2022-04-15 19:08:24+00:00,I remember back then I used to work at a smoot...,TSLA,"Tesla, Inc."
19139,2022-03-10 13:31:52+00:00,Bought some at $847. $TSLA,TSLA,"Tesla, Inc."
6411,2022-07-16 11:38:47+00:00,for the week $TSLA $NFLX $BAC $GS $SCHW $JNJ ...,TSLA,"Tesla, Inc."


In [30]:
finance_data = pd.read_csv("data/stock_yfinance_data.csv")
tweet_data = pd.read_csv("data/stock_tweets.csv")

# Ensure 'Date' columns are in datetime format and only keep the date part
finance_data['Date'] = pd.to_datetime(finance_data['Date']).dt.date
tweet_data['Date'] = pd.to_datetime(tweet_data['Date']).dt.date

# Count tweets per day for each stock
tweet_counts = tweet_data.groupby(['Date', 'Stock Name']).size().reset_index(name='Number of Tweets')

# Merge tweet_counts with finance_data on Date and Stock Name
finance_data = pd.merge(finance_data, tweet_counts, on=['Date', 'Stock Name'], how='left')
finance_data

,Date,Open,High,Low,Close,Adj Close,Volume,Stock Name,Number of Tweets
0,2021-09-30,260.333344,263.043335,258.333344,258.493347,258.493347,53868000,TSLA,90.0
1,2021-10-01,259.466675,260.260010,254.529999,258.406677,258.406677,51094200,TSLA,94.0
2,2021-10-04,265.500000,268.989990,258.706665,260.510010,260.510010,91449900,TSLA,119.0
3,2021-10-05,261.600006,265.769989,258.066681,260.196655,260.196655,55297800,TSLA,88.0
4,2021-10-06,258.733337,262.220001,257.739990,260.916656,260.916656,43898400,TSLA,78.0
...,...,...,...,...,...,...,...,...,...
6295,2022-09-23,13.090000,13.892000,12.860000,13.710000,13.710000,28279600,XPEV,NaN
6296,2022-09-26,14.280000,14.830000,14.070000,14.370000,14.370000,27891300,XPEV,1.0
6297,2022-09-27,14.580000,14.800000,13.580000,13.710000,13.710000,21160800,XPEV,NaN
6298,2022-09-28,13.050000,13.421000,12.690000,13.330000,13.330000,31799400,XPEV,NaN


# Vader

In [31]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def merge_dataframes(finance_data, sentiment_data, method):
    """
    Processes tweet data to calculate the number of tweets and average sentiment per day,
    then merges the results with finance data.

    Args:
        finance_data (pd.DataFrame): DataFrame containing finance data with 'Date' and 'Stock Name' columns.
        tweet_data (pd.DataFrame): DataFrame containing tweet data with 'Date', 'Tweet', and 'Stock Name' columns.

    Returns:
        pd.DataFrame: Merged DataFrame with additional columns for the number of tweets and average sentiment.
    """
    
        # Ensure 'Date' columns are in datetime format and only keep the date part
    finance_data['Stock Name'] = finance_data['Stock Name'].str.strip().str.upper()
    sentiment_data['Stock Name'] = sentiment_data['Stock Name'].str.strip().str.upper()
    finance_data['Date'] = pd.to_datetime(finance_data['Date']).dt.date
    sentiment_data['Date'] = pd.to_datetime(sentiment_data['Date']).dt.date

    # Group tweet data by Date and Stock Name to calculate tweet counts and average sentiment
    avg_sentiments = sentiment_data.groupby(['Date', 'Stock Name'])['sentiment'].mean().reset_index(name='Average Sentiment')
    
    # Merge finance data with tweet summary data
    merged_df = pd.merge(finance_data, avg_sentiments, on=['Date', 'Stock Name'], how='left')

    # Fill NaN values for days with no tweets
    merged_df['Number of Tweets'] = merged_df['Number of Tweets'].fillna(0)
    merged_df['Average Sentiment'] = merged_df['Average Sentiment'].fillna(0)

    # Drop rows with NaN values in critical columns
    merged_df = merged_df.dropna(subset=['Close'])
    
    merged_df.to_csv(f"data/cleaned_data_{method}.csv")

    return merged_df

# Vader

In [ ]:
tqdm.pandas()
sentiment = SentimentIntensityAnalyzer()

def get_sentiment_score(text):
    return sentiment.polarity_scores(text)['compound']

tweets_df_vader = tweets_df.copy()

tweets_df_vader['sentiment'] = tweets_df_vader['Tweet'].progress_apply(get_sentiment_score)

merge_dataframes(finance_data, tweets_df_vader, "vader")

100%|██████████| 80793/80793 [00:07<00:00, 10287.68it/s]


,Date,Open,High,Low,Close,Adj Close,Volume,Stock Name,Number of Tweets,Average Sentiment
0,2021-09-30,260.333344,263.043335,258.333344,258.493347,258.493347,53868000,TSLA,90.0,0.202088
1,2021-10-01,259.466675,260.260010,254.529999,258.406677,258.406677,51094200,TSLA,94.0,0.216879
2,2021-10-04,265.500000,268.989990,258.706665,260.510010,260.510010,91449900,TSLA,119.0,0.125718
3,2021-10-05,261.600006,265.769989,258.066681,260.196655,260.196655,55297800,TSLA,88.0,0.091361
4,2021-10-06,258.733337,262.220001,257.739990,260.916656,260.916656,43898400,TSLA,78.0,0.210363
...,...,...,...,...,...,...,...,...,...,...
6295,2022-09-23,13.090000,13.892000,12.860000,13.710000,13.710000,28279600,XPEV,0.0,0.000000
6296,2022-09-26,14.280000,14.830000,14.070000,14.370000,14.370000,27891300,XPEV,1.0,0.585900
6297,2022-09-27,14.580000,14.800000,13.580000,13.710000,13.710000,21160800,XPEV,0.0,0.000000
6298,2022-09-28,13.050000,13.421000,12.690000,13.330000,13.330000,31799400,XPEV,0.0,0.000000


# Bert

In [33]:
def is_financial_tweet(tweet):
    # Financial keywords

    financial_keywords = [
        'stock', 'market', 'trading', 'shares', 'portfolio', 'revenue',
        'profit', 'loss', 'growth', 'inflation', 'interest', 'rate', 
        'dividend', 'crypto', 'bitcoin', 'Fed', 'IPO', 'earnings', 'forecast', 
        'guidance', 'EBITDA', 'margin', 'cash flow', 'assets', 'liabilities', 
        'stock price', 'valuation', 'P/E ratio', 'EPS', 'dividend', 
        'market cap', 'volatility', 'quarterly report', 'earnings call', 
        'share buyback', 'merger', 'acquisition', 'upgrade', 'downgrade', 'estimates', '$',
        'interest', 'debt', 'decline', 'net income', 'gross income', 'operating income', 'bull', 
        'bullish', 'bear', 'bearish', 'green', 'red', 'security', 'securities'
    ]

    for i in financial_keywords:
        if i.lower() in tweet.lower():
            return True
    return False

In [ ]:
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Enable tqdm for pandas
tqdm.pandas()

# Load FinBERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model_finance = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")

# Define financial sentiment analysis pipeline
nlp_finance = pipeline("sentiment-analysis", model=model_finance, tokenizer=tokenizer)

model_general = BertForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

# Load general sentiment analysis pipeline
nlp_general = pipeline("sentiment-analysis", model=model_general, tokenizer=tokenizer)

# TODO fix this shit

# Define the function for applying sentiment analysis
def analyze_sentiment(tweet):
    if is_financial_tweet(tweet):
        score,label = nlp_finance(tweet)[0]["score"], nlp_finance(tweet)[0]["label"]
        
        if label == "Positive":
            return score
        elif label == "Negative":
            return score * -1
        else:
            return 1 - score
    else:
        score,label = nlp_general(tweet)[0]["score"], nlp_general(tweet)[0]["label"]
        if label == "5 stars":
            return score+5
        elif label == "4 stars":
            return score +4
        elif label == "3 stars":
            return score + 3
        elif label == "2 stars":
            return score +2
        else:
            return score + 1


# TODO take out the 300 when done testing
tweets_df_bert = tweets_df.copy()[:300]

# Apply the function with tqdm progress bar
tweets_df_bert['sentiment'] = tweets_df_bert['Tweet'].progress_apply(analyze_sentiment)

# tweets_df_bert
merge_dataframes(finance_data, tweets_df_bert, "bert")

100%|██████████| 300/300 [00:37<00:00,  7.90it/s]


,Date,Open,High,Low,Close,Adj Close,Volume,Stock Name,Number of Tweets,Average Sentiment
0,2021-09-30,260.333344,263.043335,258.333344,258.493347,258.493347,53868000,TSLA,90.0,0.0
1,2021-10-01,259.466675,260.260010,254.529999,258.406677,258.406677,51094200,TSLA,94.0,0.0
2,2021-10-04,265.500000,268.989990,258.706665,260.510010,260.510010,91449900,TSLA,119.0,0.0
3,2021-10-05,261.600006,265.769989,258.066681,260.196655,260.196655,55297800,TSLA,88.0,0.0
4,2021-10-06,258.733337,262.220001,257.739990,260.916656,260.916656,43898400,TSLA,78.0,0.0
...,...,...,...,...,...,...,...,...,...,...
6295,2022-09-23,13.090000,13.892000,12.860000,13.710000,13.710000,28279600,XPEV,0.0,0.0
6296,2022-09-26,14.280000,14.830000,14.070000,14.370000,14.370000,27891300,XPEV,1.0,0.0
6297,2022-09-27,14.580000,14.800000,13.580000,13.710000,13.710000,21160800,XPEV,0.0,0.0
6298,2022-09-28,13.050000,13.421000,12.690000,13.330000,13.330000,31799400,XPEV,0.0,0.0
